In [1]:
import pandas as pd
import pymongo
from sqlalchemy import create_engine
import numpy as np

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
def randomIndex(low=0,high=10,size=10):
    index_list =[]
    while len(index_list) != size:
        num=np.random.randint(low, high,1)
        if num not in index_list:
            index_list.append(num[0])
    return sorted(list((index_list)))
        
def col_to_list(col):
    return list(col.find()), len(list(col.find()))

In [4]:
quotes_mdb=client.quotes_db
author_info_col=quotes_mdb.author_information_collection


author_list,len_author_list=col_to_list(author_info_col)

author_info_df=pd.DataFrame(author_list,index=randomIndex(100,300,len_author_list))


author_info_table = author_info_df[['name','born','description']].copy()
author_info_table.index.name='author_id'
author_info_table=author_info_table.drop_duplicates(subset=['name'])
author_info_table #sql table 1

,name,born,description
author_id,,,
101,Albert Einstein,"March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
102,J.K. Rowling,"July 31, 1965 in Yate, South Gloucestershire, ...",\n See also: Robert GalbraithAlthough s...
104,Jane Austen,"December 16, 1775 in Steventon Rectory, Hampsh...",\n Jane Austen was an English novelist ...
105,Marilyn Monroe,"June 01, 1926 in The United States",\n Marilyn Monroe (born Norma Jeane Mor...
112,André Gide,"November 22, 1869 in Paris, France",\n André Paul Guillaume Gide was a Fren...
115,Thomas A. Edison,"February 11, 1847 in Milan, Ohio, The United S...",\n Thomas Alva Edison was an American i...
116,Eleanor Roosevelt,"October 11, 1884 in The United States",\n Anna Eleanor Roosevelt was an Americ...
118,Steve Martin,"August 14, 1945 in Waco, Texas, The United States","\n Stephen Glenn ""Steve"" Martin is an A..."
126,Bob Marley,"February 06, 1945 in Nine Mile, Saint Ann, Jam...","\n Robert ""Bob"" Nesta Marley OM was a J..."


In [5]:
tags_col=quotes_mdb.tags_collection
tags_list,len_tags_list=col_to_list(tags_col)
tags_df=pd.DataFrame(tags_list,index=randomIndex(301,500,len_tags_list))

tags_table = tags_df[['tag']].copy()
tags_table=tags_table.rename(columns={'tag':'tag_name'})
tags_table.index.name = 'tag_id'
tags_table #sql table 2

,tag_name
tag_id,
301,change
302,deep-thoughts
303,thinking
305,world
306,abilities
...,...
495,difficult
496,grown-ups
497,write


#### 

In [6]:
quotes_col=quotes_mdb.tag_relation_collection
quotes_list,len_quotes_list=col_to_list(quotes_col)
quotes_df= pd.DataFrame(quotes_list,index=randomIndex(501,700,len_quotes_list))
# quotes_df

In [7]:
quotes_and_tag_table = quotes_df[["quote_text",'tag']].copy()
quotes_and_tag_table=quotes_and_tag_table.rename(columns={'tag':'tag_list'})
quotes_and_tag_table.index.name = 'quote_id'
# quotes_and_tag_table

In [8]:
quoteid_tag=[]

for index, row in quotes_and_tag_table.iterrows():
    for tag in row['tag_list']:
        quoteid_tag.append((index,tag))
quote_tag_df=pd.DataFrame(quoteid_tag,columns=['quote_id','tag_name'])


In [9]:
# quote_tag_df

In [10]:
quotes_and_tag_relations_table=tags_table.reset_index().merge(quote_tag_df,on='tag_name')
quotes_and_tag_relations_table=quotes_and_tag_relations_table[['tag_id','quote_id']].copy()

In [11]:
quotes_and_tag_relations_table=quotes_and_tag_relations_table.set_index('tag_id') #sql table 3

In [12]:
quotes_table=quotes_and_tag_table[['quote_text']].copy()
# quotes_table

In [13]:
quotes_col=quotes_mdb.quotes_collection
quotes_and_author_df=pd.DataFrame(list(quotes_col.find()))
quotes_and_author_table=quotes_and_author_df[['quote_text','author_name']].copy().rename(columns={'author_name':'name'})

In [14]:
quotes_and_author_table=quotes_and_author_table.merge(author_info_table.reset_index(), on='name')
# quotes_and_author_table

In [15]:
quotes_and_author_table=(quotes_and_author_table[['quote_text','author_id']].copy()).merge(quotes_and_tag_table.reset_index(),on="quote_text").drop(['tag_list'],axis=1)

In [16]:
quotes_and_author_table=quotes_and_author_table.set_index('quote_id')

In [17]:
quotes_and_author_table

,quote_text,author_id
quote_id,,
501,“The world as we have created it is a process ...,101
507,“There are only two ways to live your life. On...,101
515,“Try not to become a man of success. Rather be...,101
532,"“If you can't explain it to a six year old, yo...",101
553,"“If you want your children to be intelligent, ...",101
...,...,...
681,“To die will be an awfully big adventure.”,277
683,“It takes courage to grow up and become who yo...,278
689,“But better to get hurt by the truth than comf...,279


In [18]:
pword = input("enter your postgres password: ")

rds_connection_string = f"postgres:{pword}@localhost:5432/Quotes_DB"#<=== username is postgres and mypassword is next
engine = create_engine(f'postgresql://{rds_connection_string}')

# Confirm tables
engine.table_names()

enter your postgres password:  Baggins89


['quote_tag_relation', 'tags', 'author_info', 'quotes']

In [24]:
overwrite='append'

In [26]:
author_info_table.to_sql(name='author_info',con=engine,if_exists='append',index=True)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "author_info_pkey"
DETAIL:  Key (author_id)=(101) already exists.

[SQL: INSERT INTO author_info (author_id, name, born, description) VALUES (%(author_id)s, %(name)s, %(born)s, %(description)s)]
[parameters: ({'author_id': 101, 'name': 'Albert Einstein', 'born': 'March 14, 1879 in Ulm, Germany', 'description': '\n        In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining th ... (3554 characters truncated) ... have made the word "Einstein" synonymous with genius.More: http://en.wikipedia.org/wiki/Albert_E...http://www.nobelprize.org/nobel_prize...    \n    '}, {'author_id': 102, 'name': 'J.K. Rowling', 'born': 'July 31, 1965 in Yate, South Gloucestershire, England, The United Kingdom', 'description': '\n        See also: Robert GalbraithAlthough she writes under the pen name J.K. Rowling, pronounced like ro

In [21]:
quotes_and_author_table.to_sql(name='quotes',con=engine,if_exists=overwrite,index=True)

In [22]:
tags_table.to_sql(name='tags',con=engine,if_exists=overwrite,index=True)

In [23]:
quotes_and_tag_relations_table.to_sql(name='quote_tag_relation',con=engine,if_exists=overwrite,index=True)